# Bibliotecas

In [ ]:
# Caminhos
import utilitarios.caminhos as caminho

# Download dados
from utilitarios.download_dos_dados import download_dados

# Regressão linear
from utilitarios.modelo_regressao import regressao_linear, predicao_regressao

# Rede neural
from utilitarios.modelo_rede_neural import rede_neural, predicao_rede_neural

# Gerar imagens
from utilitarios.gerar_imagens import histograma, grafico_regressao_univariada, histograma_dos_erros, grafico_3d_multiplos_angulos

# Transformações dados
from utilitarios.transformacoes_dados import normalizacao, desnormalizar

# Funções estatisticas
from utilitarios.estatisticas import calcular_r_quadrado, criar_tabela, para_dataframe

# Outros
from pathlib import Path
from random import sample, seed, random
seed(2025)

# Dados

## Baixando dados

In [ ]:
dados = download_dados()
dados = dados.rename(columns = {"soc_can":"soc",
                                "soh_can":"soh"})

colunas_dados:list = dados.columns
print(f"Quantidade de dados: {len(dados)}")
dados.head(5)

## Estatísticas descritivas

In [ ]:
dados.describe().to_csv(Path(caminho.tabelas, "resumo_descritivo.csv"))
dados.describe()

## Separando dados em treino e teste

In [ ]:
proporcao_treino:float = 0.8
proporcao_teste:float = 1 - proporcao_treino

amostras_treino:int = int(proporcao_treino*len(dados))
amostras_teste:int = len(dados) - amostras_treino

lista_amostras_treino:list = sample(range(amostras_treino), amostras_treino)

dados_treino = dados.iloc[lista_amostras_treino].iloc[:2000]
dados_teste = dados.drop(index=lista_amostras_treino).sort_index().iloc[:1000]

print(f"Total de amostras de treino: {amostras_treino}\nTotal de amostras de teste: {amostras_teste}")

## Parâmetros globais dos dados

In [ ]:
media_dados_treino:dict = {}
desvio_padrao_dados_treino:dict = {}

for coluna in dados_treino.columns:
    media_dados_treino[coluna] = dados_treino[coluna].mean()
    desvio_padrao_dados_treino[coluna] = dados_treino[coluna].std()


media_dados_teste:dict = {}
desvio_padrao_dados_teste:dict = {}

for coluna in dados_teste.columns:
    media_dados_teste[coluna] = dados_teste[coluna].mean()
    desvio_padrao_dados_teste[coluna] = dados_teste[coluna].std()

## Distribuição dos dados

In [ ]:
for coluna in colunas_dados:
    histograma(dados = dados, coluna = coluna,
               salvar_em = Path(caminho.imagens_dados, f"distribuicao_dos_dados_de_{coluna}.png"),
               bins = 16)

# Parâmetros para regressões

In [ ]:
regressao_univariada:list = [{"X":["voltage"], "y":"power"},
                             {"X":["current"], "y":"power"},
                             {"X":["speed"], "y":"power"},
                             {"X":["latitude"], "y":"power"},
                             {"X":["longitude"], "y":"power"},
                             {"X":["soc"], "y":"power"},]

regressao_multivariada:list = [{"X":["current", "voltage", "speed"] , "y":"power"},
                               {"X":["current", "speed"] , "y":"power"},
                               {"X":["latitude", "longitude", "speed"] , "y":"power"},
                               {"X":["latitude", "longitude"] , "y":"power"},
                               {"X":["latitude", "speed"] , "y":"power"},
                               {"X":["longitude", "speed"] , "y":"power"},
                               {"X":["voltage", "latitude", "longitude", "speed"] , "y":"power"}]

globais:dict = {}

criar_tabela(regressao_univariada, nome = "regressoes_univariadas.csv")
criar_tabela(regressao_multivariada, nome = "regressoes_multivariadas.csv")

# Regressão Linear clássica

## Univariada

In [ ]:
for index, parametros in enumerate(regressao_univariada):
    regressao_univariada[index]["modelo_regressao"] = regressao_linear(dados = dados_treino, 
                                                                       X = parametros["X"], 
                                                                       y = parametros["y"])

    regressao_univariada[index]["y_treino_regressao"] = predicao_regressao(dados = dados_treino,
                                                                           modelo = parametros["modelo_regressao"],
                                                                           X = parametros["X"])

    residuos = regressao_univariada[index]["modelo_regressao"].resid
    mse = (residuos**2).mean()
    regressao_univariada[index]["mse_regressao"] = mse

    grafico_regressao_univariada(dados = dados_treino,
                                 X = parametros["X"],
                                 y = parametros["y"],
                                 real_y = regressao_univariada[index]["y_treino_regressao"],
                                 salvar_em = Path(caminho.imagens_regressao_previsao, f"dados_treino_{parametros["X"][0]}_{parametros["y"]}"))

## Multivariada

In [ ]:
for index, parametros in enumerate(regressao_multivariada):
    regressao_multivariada[index]["modelo_regressao"] = regressao_linear(dados = dados_treino, 
                                                                         X = parametros["X"], 
                                                                         y = parametros["y"])

    residuos = regressao_multivariada[index]["modelo_regressao"].resid
    mse = (residuos**2).mean()
    regressao_multivariada[index]["mse_regressao"] = mse

# Redes neurais

## Univariada

In [ ]:
for index, parametros in enumerate(regressao_univariada):
    print(f"\rTreinando rede {index}/{len(regressao_univariada)}", end = "")
    regressao_univariada[index]["modelo_rede_neural"] = rede_neural(dados = normalizacao(dados_treino,
                                                                                         media = media_dados_treino,
                                                                                         desvio_padrao = desvio_padrao_dados_treino), 
                                                                    X = parametros["X"], 
                                                                    y = parametros["y"])

    regressao_univariada[index]["y_treino_rede_neural"] = predicao_rede_neural(dados = normalizacao(dados_treino,
                                                                                                    media = media_dados_treino,
                                                                                                    desvio_padrao = desvio_padrao_dados_treino), 
                                                                               modelo = parametros["modelo_rede_neural"], 
                                                                               X = parametros["X"])

    regressao_univariada[index]["y_treino_rede_neural"] = desnormalizar(regressao_univariada[index]["y_treino_rede_neural"],
                                                                        media = media_dados_treino[parametros["y"]],
                                                                        desvio_padrao = desvio_padrao_dados_treino[parametros["y"]])

    regressao_univariada[index]["mse_rede_neural"] = 1/len(dados_treino[parametros["y"]]) * float(sum([(real - estimado)**2 for real, estimado in zip(dados_treino[parametros["y"]], regressao_univariada[index]["y_treino_rede_neural"])]))

    grafico_regressao_univariada(dados = dados_treino,
                                 X = parametros["X"],
                                 y = parametros["y"],
                                 real_y = regressao_univariada[index]["y_treino_rede_neural"],
                                 salvar_em = Path(caminho.imagens_rede_neural_previsao, f"dados_treino_{parametros["X"][0]}_{parametros["y"]}"))


print(f"\rTreinando rede {len(regressao_univariada)}/{len(regressao_univariada)}")

## Multivariada

In [ ]:
for index, parametros in enumerate(regressao_multivariada):
    print(f"\rTreinando rede {index}/{len(regressao_multivariada)}", end = "")
    regressao_multivariada[index]["modelo_rede_neural"] = rede_neural(dados = normalizacao(dados_treino,
                                                                                           media = media_dados_treino,
                                                                                           desvio_padrao = desvio_padrao_dados_treino), 
                                                                      X = parametros["X"], 
                                                                      y = parametros["y"])

    regressao_multivariada[index]["y_treino_rede_neural"] = predicao_rede_neural(dados = normalizacao(dados_treino,
                                                                                                       media = media_dados_treino,
                                                                                                       desvio_padrao = desvio_padrao_dados_treino), 
                                                                                  modelo = parametros["modelo_rede_neural"],
                                                                                  X = parametros["X"])

    regressao_multivariada[index]["y_treino_rede_neural"] = desnormalizar(regressao_multivariada[index]["y_treino_rede_neural"],
                                                                           media = media_dados_treino[parametros["y"]],
                                                                           desvio_padrao = desvio_padrao_dados_treino[parametros["y"]])

    regressao_multivariada[index]["mse_rede_neural"] = 1/len(dados_treino[parametros["y"]]) * float(sum([(real - estimado)**2 for real, estimado in zip(dados_treino[parametros["y"]], regressao_multivariada[index]["y_treino_rede_neural"])]))


    
print(f"\rTreinando rede {len(regressao_multivariada)}/{len(regressao_multivariada)}")

# Resultados

## Imagens

### Previsão Regressão Clássica

In [ ]:
for index, parametros in enumerate(regressao_univariada):
    regressao_univariada[index]["y_predito_regressao"] = predicao_regressao(dados = dados_teste, modelo = parametros["modelo_regressao"], X = parametros["X"])

    grafico_regressao_univariada(dados = dados_teste,
                                 X = parametros["X"],
                                 y = parametros["y"],
                                 real_y = regressao_univariada[index]["y_predito_regressao"],
                                 salvar_em = Path(caminho.imagens_regressao_previsao, f"dados_teste_{parametros['X'][0]}_{parametros['y']}"))

    regressao_univariada[index]["erros_regressao"] = [float(real - predito) for real, predito in zip(dados_teste[parametros["y"]], regressao_univariada[index]["y_predito_regressao"])]
    

for index, parametros in enumerate(regressao_multivariada):
    regressao_multivariada[index]["y_predito_regressao"] = predicao_regressao(dados = dados_teste,
                                                                              modelo = parametros["modelo_regressao"], 
                                                                              X = parametros["X"])

    regressao_multivariada[index]["erros_regressao"] = [float(real - predito) for real, predito in zip(dados_teste[parametros["y"]], regressao_multivariada[index]["y_predito_regressao"])]

### Previsão Redes Neurais

In [ ]:
for index, parametros in enumerate(regressao_univariada):
    regressao_univariada[index]["y_predito_rede_neural"] = predicao_rede_neural(dados = normalizacao(dados_teste,
                                                                                                     media = media_dados_teste,
                                                                                                     desvio_padrao = desvio_padrao_dados_teste), 
                                                                                modelo = parametros["modelo_rede_neural"],
                                                                                X = parametros["X"])

    regressao_univariada[index]["y_predito_rede_neural"] = desnormalizar(regressao_univariada[index]["y_predito_rede_neural"],
                                                                         media = media_dados_teste[parametros["y"]],
                                                                         desvio_padrao = desvio_padrao_dados_teste[parametros["y"]])

    regressao_univariada[index]["erros_rede_neural"] = [float(real - predito) for real, predito in zip(dados_teste[parametros["y"]], regressao_univariada[index]["y_predito_rede_neural"])]
    
    grafico_regressao_univariada(dados = dados_teste,
                                 X = parametros["X"],
                                 y = parametros["y"],
                                 real_y = regressao_univariada[index]["y_predito_rede_neural"],
                                 salvar_em = Path(caminho.imagens_rede_neural_previsao, f"dados_teste_{parametros['X'][0]}_{parametros['y']}"))


for index, parametros in enumerate(regressao_multivariada):
    regressao_multivariada[index]["y_predito_rede_neural"] = predicao_rede_neural(dados = normalizacao(dados_teste,
                                                                                                       media = media_dados_teste,
                                                                                                       desvio_padrao = desvio_padrao_dados_teste), 
                                                                                  modelo = parametros["modelo_rede_neural"],
                                                                                  X = parametros["X"])

    regressao_multivariada[index]["y_predito_rede_neural"] = desnormalizar(regressao_multivariada[index]["y_predito_rede_neural"],
                                                                           media = media_dados_teste[parametros["y"]],
                                                                           desvio_padrao = desvio_padrao_dados_teste[parametros["y"]])


    regressao_multivariada[index]["erros_rede_neural"] = [float(real - predito) for real, predito in zip(dados_teste[parametros["y"]], regressao_multivariada[index]["y_predito_rede_neural"])]

### Histograma dos erros para Regressão Clássica

In [ ]:
for index, parametros in enumerate(regressao_univariada):
    histograma_dos_erros(dados = regressao_univariada[index]["erros_regressao"],
                             salvar_em = Path(caminho.imagens_regressao_histograma, f"distribuicao_dos_erros_de_{'+'.join(parametros['X'])}_{parametros['y']}.png"),
                             nome_coluna = f"f({', '.join(parametros['X'])}) -> {parametros['y']}",
                             bins = 24)


for index, parametros in enumerate(regressao_multivariada):
    histograma_dos_erros(dados = regressao_multivariada[index]["erros_regressao"],
                         salvar_em = Path(caminho.imagens_regressao_histograma, f"distribuicao_dos_erros_de_{'+'.join(parametros['X'])}_{parametros['y']}.png"),
                         nome_coluna = f"f({', '.join(parametros['X'])}) -> {parametros['y']}",
                         bins = 24)

### Histograma dos erros para Redes Neurais

In [ ]:
for index, parametros in enumerate(regressao_univariada):
    histograma_dos_erros(dados = regressao_univariada[index]["erros_rede_neural"],
                         salvar_em = Path(caminho.imagens_rede_neural_histograma, f"distribuicao_dos_erros_de_{'+'.join(parametros['X'])}_{parametros['y']}.png"),
                         nome_coluna = f"f({', '.join(parametros['X'])}) -> {parametros['y']}",
                         bins = 24)


for index, parametros in enumerate(regressao_multivariada):
    histograma_dos_erros(dados = regressao_multivariada[index]["erros_rede_neural"],
                         salvar_em = Path(caminho.imagens_rede_neural_histograma, f"distribuicao_dos_erros_de_{'+'.join(parametros['X'])}_{parametros['y']}.png"),
                         nome_coluna = f"f({', '.join(parametros['X'])}) -> {parametros['y']}",
                         bins = 24)

### Vizualização 3d

In [ ]:
grafico_3d_multiplos_angulos(dados = dados_treino, x = "current", y = "speed", z = "power")
grafico_3d_multiplos_angulos(dados = dados_treino, x = "latitude", y = "longitude", z = "power")

## Tabelas 

### $\text{R}^2$

In [ ]:
tabela = calcular_r_quadrado(dados = dados_treino,
                             dicionario_memoria = regressao_univariada,
                             nome_complementar = "_univariada")
tabela

In [ ]:
tabela = calcular_r_quadrado(dados = dados_treino,
                             dicionario_memoria = regressao_multivariada,
                             nome_complementar = "_multivariada")
tabela

# Exemplos Artificiais

## Exemplo 1

In [ ]:
modelos_artificiais:dict = {}

In [ ]:
y:list = [0, 1, 2, 3, 4, 5, 5, 5, 5, 5, 6, 7, 8, 9, 10]
X:list = [0, 0, 0, 0, 0, *[i for i in range(len(y))], 15, 15, 15, 15, 15]
y:list = [0, 0, 0, 0, 0, *y, 10, 10, 10, 10, 10]
tabela_teste = para_dataframe({"X_r2_mse":X*20,
                               "y_r2_mse":y*20})


for coluna in tabela_teste.columns: 
    for index in range(len(tabela_teste[coluna])):
        tabela_teste[coluna].iloc[index] = (tabela_teste[coluna].iloc[index] + random()*1.2)
    media, dp = tabela_teste[coluna].mean(), tabela_teste[coluna].std()
    tabela_teste[coluna] = (tabela_teste[coluna] - media)/dp 

In [ ]:
modelos_artificiais["r2_mse_regressao"] = {}

modelos_artificiais["r2_mse_regressao"]["modelo"] = regressao_linear(dados = tabela_teste, 
                                                                     X = ["X_r2_mse"], 
                                                                     y = "y_r2_mse")

modelos_artificiais["r2_mse_regressao"]["y_treino"] = predicao_regressao(dados = tabela_teste,
                                                                         modelo = modelos_artificiais["r2_mse_regressao"]["modelo"],
                                                                         X = ["X_r2_mse"])

residuos = modelos_artificiais["r2_mse_regressao"]["modelo"].resid
mse = (residuos**2).mean()
modelos_artificiais["r2_mse_regressao"]["mse"] = mse

residuos = modelos_artificiais["r2_mse_regressao"]["modelo"].resid

grafico_regressao_univariada(dados = tabela_teste,
                             X = ["X_r2_mse"],
                             y = "y_r2_mse",
                             real_y = modelos_artificiais["r2_mse_regressao"]["y_treino"],
                             salvar_em = Path(caminho.imagens_regressao_previsao, f"dados_treino_X_r2_mse_y_r2_mse"))

In [ ]:
modelos_artificiais["r2_mse_rede_neural"] = {}

modelos_artificiais["r2_mse_rede_neural"]["modelo"] = rede_neural(dados = tabela_teste,
                                                                  X = ["X_r2_mse"], 
                                                                  y = "y_r2_mse",
                                                                  neuronios = 16)

modelos_artificiais["r2_mse_rede_neural"]["y_treino"] = predicao_rede_neural(dados = tabela_teste,
                                                                             modelo = modelos_artificiais["r2_mse_rede_neural"]["modelo"],
                                                                             X = ["X_r2_mse"])

modelos_artificiais["r2_mse_regressao"]["erros"] = [(real - estimado)**2 for real, estimado in zip(tabela_teste["y_r2_mse"], modelos_artificiais["r2_mse_rede_neural"]["y_treino"])]

modelos_artificiais["r2_mse_rede_neural"]["mse"] = 1/len(tabela_teste["y_r2_mse"]) * float(sum(modelos_artificiais["r2_mse_regressao"]["erros"]))

grafico_regressao_univariada(dados = tabela_teste,
                             X = ["X_r2_mse"],
                             y = "y_r2_mse",
                             real_y = modelos_artificiais["r2_mse_rede_neural"]["y_treino"],
                             salvar_em = Path(caminho.imagens_rede_neural_previsao, f"dados_treino_X_r2_mse_y_r2_mse"))

In [ ]:
media = tabela_teste["y_r2_mse"].mean()

modelos_artificiais["r2_mse_regressao"]["r^2"] = 1 - sum([i*i for i in modelos_artificiais["r2_mse_regressao"]["modelo"].resid])/sum([(real - media)*(real - media) for real in tabela_teste["y_r2_mse"]])
modelos_artificiais["r2_mse_rede_neural"]["r^2"] = 1 - float(sum([i*i for i in modelos_artificiais["r2_mse_regressao"]["erros"]])/sum([(real - media)*(real - media) for real in tabela_teste["y_r2_mse"]]))

print(f'Regressão linear:\n\tMSE: {modelos_artificiais["r2_mse_regressao"]["mse"]:0.04f} | R^2: {modelos_artificiais["r2_mse_regressao"]["r^2"]:0.04f}')
print(f'Rede Neural:\n\tMSE: {modelos_artificiais["r2_mse_rede_neural"]["mse"]:0.04f} {modelos_artificiais["r2_mse_rede_neural"]["r^2"]:0.04f}')

## Exemplo 2

In [ ]:
modelos_artificiais:dict = {}

In [ ]:
y:list = [0, 1, 2, 3, 4, 5, 6, 5, 4, 3, 2 , 1]
X:list = [i for i in range(len(y))]
tabela_teste = para_dataframe({"X":X*20,
                               "y":y*20})


for coluna in tabela_teste.columns: 
    for index in range(len(tabela_teste[coluna])):
        tabela_teste[coluna].iloc[index] = (tabela_teste[coluna].iloc[index] + random()*1.2)
    media, dp = tabela_teste[coluna].mean(), tabela_teste[coluna].std()
    tabela_teste[coluna] = (tabela_teste[coluna] - media)/dp 

In [ ]:
modelos_artificiais["r2_mse_regressao"] = {}

modelos_artificiais["r2_mse_regressao"]["modelo"] = regressao_linear(dados = tabela_teste, 
                                                                     X = ["X"], 
                                                                     y = "y")

modelos_artificiais["r2_mse_regressao"]["y_treino"] = predicao_regressao(dados = tabela_teste,
                                                                         modelo = modelos_artificiais["r2_mse_regressao"]["modelo"],
                                                                         X = ["X"])

residuos = modelos_artificiais["r2_mse_regressao"]["modelo"].resid
mse = (residuos**2).mean()
modelos_artificiais["r2_mse_regressao"]["mse"] = mse

residuos = modelos_artificiais["r2_mse_regressao"]["modelo"].resid

grafico_regressao_univariada(dados = tabela_teste,
                             X = ["X"],
                             y = "y",
                             real_y = modelos_artificiais["r2_mse_regressao"]["y_treino"],
                             salvar_em = Path(caminho.imagens_regressao_previsao, f"dados_treino_X_y"))

In [ ]:
modelos_artificiais["r2_mse_rede_neural"] = {}

modelos_artificiais["r2_mse_rede_neural"]["modelo"] = rede_neural(dados = tabela_teste,
                                                                  X = ["X"], 
                                                                  y = "y",
                                                                  neuronios = 2)

modelos_artificiais["r2_mse_rede_neural"]["y_treino"] = predicao_rede_neural(dados = tabela_teste,
                                                                             modelo = modelos_artificiais["r2_mse_rede_neural"]["modelo"],
                                                                             X = ["X"])

modelos_artificiais["r2_mse_regressao"]["erros"] = [(real - estimado)**2 for real, estimado in zip(tabela_teste["y"], modelos_artificiais["r2_mse_rede_neural"]["y_treino"])]

modelos_artificiais["r2_mse_rede_neural"]["mse"] = 1/len(tabela_teste["y"]) * float(sum(modelos_artificiais["r2_mse_regressao"]["erros"]))

grafico_regressao_univariada(dados = tabela_teste,
                             X = ["X"],
                             y = "y",
                             real_y = modelos_artificiais["r2_mse_rede_neural"]["y_treino"],
                             salvar_em = Path(caminho.imagens_rede_neural_previsao, f"dados_treino_X_y"))